## Preamble

In [ ]:
# Required packages
import numpy as np
import cv2 
from scipy.signal import savgol_filter
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from math import *
import tifffile
import warnings
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from scipy.optimize import curve_fit
import string
from pylab import *
warnings.filterwarnings("ignore")

In [ ]:
# Plot formatting 
matplotlib.rcParams['mathtext.fontset']='cm'
matplotlib.rcParams['font.family']='STIXGeneral'
plt.rcParams['legend.fontsize']=15
plt.rcParams.update({'font.size':15}) 
plt.rcParams['axes.axisbelow']=True

## Function that plots the crack length distribution

In [ ]:
def plot_lenght_filter(f_r_mm):
    '''
    f_r_mm: float -- filter length in mm

    '''
    nbins=40

    #### RESIDUAL TRHESHOLDING ####
    select = list(range(0,37,1))
    colors = plt.cm.viridis(np.linspace(0,1,37))
    cmap = plt.get_cmap('viridis', 37)
    for i in select:
        df1 = pd.read_csv('Residual_Thresholding_Fiji_output/shear_band/'+str(i)+'-nt_s2_roi.csv', delimiter=',')
        df2 = pd.read_csv('Residual_Thresholding_Fiji_output/whole_image/'+str(i)+'-nt_s2.csv', delimiter=',')
        
        ##Convert to mm 
        df1['Major adjusted']=df1['Major']*0.00791/0.001
        df2['Major adjusted']=df2['Major']*0.00791/0.001

        ##Filter DIAMETER length below 10 pixels (in mm)
        df1 = df1[df1['Major adjusted'] >= f_r_mm]
        df2 = df2[df2['Major adjusted'] >= f_r_mm]

        ##Histogram
        counts1, bins1 = np.histogram(df1['Major adjusted'], bins=nbins)
        counts2, bins2 = np.histogram(df2['Major adjusted'], bins=nbins)

        bin_centres1 = 0.5*(bins1[1:]+bins1[:-1])
        bin_centres2 = 0.5*(bins2[1:]+bins2[:-1])

        ax[0,0].plot(np.log10(bin_centres1), counts1,'o', c=colors[i],  label=str(i))
        ax[0,1].plot(np.log10(bin_centres2), counts2,'o', c=colors[i], label=str(i))

    #### SKELETONISATION ####
    selectE = list(range(0,37,1))
    colorsE = plt.cm.viridis(np.linspace(0,1,37))
    cmapE = plt.get_cmap('viridis', 37)

    for e in selectE:
        
        df1 = pd.read_csv('Skeletonisation_Fiji_output/shear_band/'+str(e)+'-s1_lukeout_roi.csv', delimiter=',')
        df2 = pd.read_csv('Skeletonisation_Fiji_output/whole_image/'+str(e)+'-s1_lukeout.csv', delimiter=',')

        ##Convert units
        df1['Major adjusted']=df1['Major']*0.00791/0.001 #convert to mm 
        df2['Major adjusted']=df2['Major']*0.00791/0.001

        ##Filter length below 10 pixels (in mm)
        df1 = df1[df1['Major adjusted'] >= f_r_mm]
        df2 = df2[df2['Major adjusted'] >= f_r_mm]

        ##Histogram
        counts1, bins1 = np.histogram(df1['Major adjusted'], bins=nbins)
        counts2, bins2 = np.histogram(df2['Major adjusted'], bins=nbins)

        bin_centres1 = 0.5*(bins1[1:]+bins1[:-1])
        bin_centres2 = 0.5*(bins2[1:]+bins2[:-1])

        ax[1,0].plot(np.log10(bin_centres1), counts1,'o', c=colors[e],  label=str(e))
        ax[1,1].plot(np.log10(bin_centres2), counts2,'o', c=colors[e], label=str(e))


    ##COLOUR BAR
    # Normalizer
    normE = mpl.colors.Normalize(vmin=0, vmax=37) 
    # creating ScalarMappable
    smE = plt.cm.ScalarMappable(cmap=cmap, norm=normE)
    smE.set_array([])
    cbE = plt.colorbar(smE, ticks=np.linspace(0,37,38), ax=ax, aspect=35)

    for label in cbE.ax.yaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    cbE.set_label('Image Number')


    ##LABELS SUBPLOTS
    for x in np.ravel(ax):
        x.set(yscale="log")
        x.set_xlabel(r'log$_{10}(l)$ ($\mu$m)')
        x.set_ylabel('Frequency')
        x.set_ylim(0.9, 10000)
        x.set_xlim(1, 3)
        x.set_xticks(np.arange(1,3,0.5))
        x.grid()
      

    #### Label Subplots
    for n, a in enumerate(ax.flat):
        a.text(-0.15, 0.98, string.ascii_lowercase[n]+")", transform = a.transAxes, size=22, weight = 'bold')
    plt.show()  

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(17,7))
bottom, top = 0, 1
left, right = 0, 0.9
fig.subplots_adjust(top = top, bottom = bottom, left = left, right = right, hspace = 0.3, wspace = 0.2)
plot_lenght_filter(0.0) # no filter 